In [5]:
# =============================================================================
# 1. 필요한 라이브러리 불러오기
# =============================================================================
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report


# 벡터화 함수
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# 머신러닝 모델들
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

# 모델 검증
from sklearn.metrics import accuracy_score #정확도 계산
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import tensorflow as tf


In [ ]:

# 반복할 num_words 옵션
num_words_options = [10000, 15000]
ml_results = {}  # 전체 결과를 저장할 딕셔너리

for nw in num_words_options:
    print("===================================================")
    print(f"[실험] num_words = {nw}")
    
    # 1) 데이터 로드
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=nw, test_split=0.2)

    # 2) 단어 사전 로드 및 인덱스→단어 매핑
    word_index = reuters.get_word_index(path="reuters_word_index.json")
    index_to_word = {index + 3: word for word, index in word_index.items()}
    for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
        index_to_word[index] = token

    # 3) 정수 시퀀스를 텍스트로 복원
    x_train_text = [' '.join([index_to_word.get(idx, "<unk>") for idx in seq]) for seq in x_train]
    x_test_text  = [' '.join([index_to_word.get(idx, "<unk>") for idx in seq]) for seq in x_test]

    # 문장 길이 통계
    lengths = [len(txt.split()) for txt in x_train_text]
    max_length = np.max(lengths)
    avg_length = np.mean(lengths)
    print(f"  문장의 최대 길이: {max_length}, 평균 길이: {avg_length:.2f}")

    # 4) TF-IDF 전처리
    dtmvector = CountVectorizer()
    x_train_dtm = dtmvector.fit_transform(x_train_text)
    tfidf_transformer = TfidfTransformer()
    tfidfv = tfidf_transformer.fit_transform(x_train_dtm)

    x_test_dtm = dtmvector.transform(x_test_text)
    tfidfv_test = tfidf_transformer.transform(x_test_dtm)

    # 5) 모델 학습 및 평가 (Accuracy, F1-score)
    #    모델별 결과(f1-score)를 저장할 딕셔너리
    experiment_result = {}

    # (a) Multinomial Naive Bayes
    print("\n[1] Multinomial Naive Bayes")
    model_mnb = MultinomialNB()
    model_mnb.fit(tfidfv, y_train)
    predicted_mnb = model_mnb.predict(tfidfv_test)
    acc_mnb = accuracy_score(y_test, predicted_mnb)
    f1_mnb = f1_score(y_test, predicted_mnb, average='weighted')
    print("  정확도:", acc_mnb)
    print("  F1-Score:", f1_mnb)
    experiment_result['MultinomialNB'] = f1_mnb

    # (b) Complement Naive Bayes
    print("\n[2] Complement Naive Bayes")
    model_cnb = ComplementNB()
    model_cnb.fit(tfidfv, y_train)
    predicted_cnb = model_cnb.predict(tfidfv_test)
    acc_cnb = accuracy_score(y_test, predicted_cnb)
    f1_cnb = f1_score(y_test, predicted_cnb, average='weighted')
    print("  정확도:", acc_cnb)
    print("  F1-Score:", f1_cnb)
    experiment_result['ComplementNB'] = f1_cnb

    # (c) Logistic Regression
    print("\n[3] Logistic Regression ")
    model_logreg = LogisticRegression(C=10000, penalty='l2', max_iter=1000, random_state=42)
    model_logreg.fit(tfidfv, y_train)
    predicted_logreg = model_logreg.predict(tfidfv_test)
    acc_logreg = accuracy_score(y_test, predicted_logreg)
    f1_logreg = f1_score(y_test, predicted_logreg, average='weighted')
    print("  정확도:", acc_logreg)
    print("  F1-Score:", f1_logreg)
    experiment_result['LogisticRegression'] = f1_logreg

    # (d) LinearSVC (
    print("\n[4] Support Vector Machine ")
    model_svc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False, random_state=42)
    model_svc.fit(tfidfv, y_train)
    predicted_svc = model_svc.predict(tfidfv_test)
    acc_svc = accuracy_score(y_test, predicted_svc)
    f1_svc = f1_score(y_test, predicted_svc, average='weighted')
    print("  정확도:", acc_svc)
    print("  F1-Score:", f1_svc)
    experiment_result['LinearSVC'] = f1_svc

    # (e) Decision Tree
    print("\n[5] Decision Tree ")
    model_tree = DecisionTreeClassifier(max_depth=10, random_state=42)
    model_tree.fit(tfidfv, y_train)
    predicted_tree = model_tree.predict(tfidfv_test)
    acc_tree = accuracy_score(y_test, predicted_tree)
    f1_tree = f1_score(y_test, predicted_tree, average='weighted')
    print("  정확도:", acc_tree)
    print("  F1-Score:", f1_tree)
    experiment_result['DecisionTree'] = f1_tree

    # (f) Random Forest
    print("\n[6] Random Forest")
    model_forest = RandomForestClassifier(n_estimators=5, random_state=42)
    model_forest.fit(tfidfv, y_train)
    predicted_forest = model_forest.predict(tfidfv_test)
    acc_forest = accuracy_score(y_test, predicted_forest)
    f1_forest = f1_score(y_test, predicted_forest, average='weighted')
    print("  정확도:", acc_forest)
    print("  F1-Score:", f1_forest)
    experiment_result['RandomForest'] = f1_forest

    # (g) Gradient Boosting
    print("\n[7] Gradient Boosting")
    model_gbt = GradientBoostingClassifier(random_state=0, verbose=0)
    model_gbt.fit(tfidfv, y_train)
    predicted_gbt = model_gbt.predict(tfidfv_test)
    acc_gbt = accuracy_score(y_test, predicted_gbt)
    f1_gbt = f1_score(y_test, predicted_gbt, average='weighted')
    print("  정확도:", acc_gbt)
    print("  F1-Score:", f1_gbt)
    experiment_result['GradientBoosting'] = f1_gbt

    # (h) Voting Classifier
    print("\n[8] Voting Classifier ")
    voting_classifier = VotingClassifier(
        estimators=[
            ('lr', LogisticRegression(C=10000, penalty='l2')),
            ('cb', ComplementNB()),
            ('grbt', GradientBoostingClassifier(random_state=0))
        ],
        voting='soft'
    )
    voting_classifier.fit(tfidfv, y_train)
    predicted_vote = voting_classifier.predict(tfidfv_test)
    acc_vote = accuracy_score(y_test, predicted_vote)
    f1_vote = f1_score(y_test, predicted_vote, average='weighted')
    print("  정확도:", acc_vote)
    print("  F1-Score:", f1_vote)
    experiment_result['Voting'] = f1_vote

    # 최종 결과 저장
    ml_results[str(nw)] = experiment_result

# 6) 전체 결과 요약
print("\n===================================================")
print("전체 머신러닝 실험 결과 (weighted f1-score):")
for nw, res_dict in ml_results.items():
    print(f"\n>>> num_words = {nw}")
    for model_name, f1_val in res_dict.items():
        print(f"  {model_name}: {f1_val:.4f}")



[실험] num_words = 10000
  문장의 최대 길이: 2376, 평균 길이: 145.54

[1] Multinomial Naive Bayes
  정확도: 0.6567230632235085
  F1-Score: 0.5764467518778252

[2] Complement Naive Bayes
  정확도: 0.7707034728406055
  F1-Score: 0.7456682614453047

[3] Logistic Regression (L2, random_state=42)
  정확도: 0.7951914514692787
  F1-Score: 0.7727935299669956

[4] Support Vector Machine (LinearSVC, random_state=42)
  정확도: 0.8299198575244879
  F1-Score: 0.8236882254849623

[5] Decision Tree (max_depth=10, random_state=42)
  정확도: 0.6219946571682992
  F1-Score: 0.5787746084604964

[6] Random Forest (n_estimators=5, random_state=42)
  정확도: 0.6776491540516474
  F1-Score: 0.6448193857300015

[7] Gradient Boosting (n_estimators=100, random_state=42)
  정확도: 0.7707034728406055
  F1-Score: 0.7664689841505945

[8] Voting Classifier (Soft Voting)
  정확도: 0.7983081032947462
  F1-Score: 0.7935713406786632
[실험] num_words = 15000
  문장의 최대 길이: 2376, 평균 길이: 145.54

[1] Multinomial Naive Bayes
  정확도: 0.6331255565449688
  F1-Score: 0.54

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score, classification_report


def tensorflow_dataset(input_x, input_y, buffer_size, batch_size):
    tf_data = tf.data.Dataset.from_tensor_slices((input_x, input_y))
    tf_data = tf_data.shuffle(buffer_size)
    tf_data = tf_data.repeat()
    tf_data = tf_data.batch(batch_size)
    tf_data = tf_data.prefetch(buffer_size=-1)
    return tf_data 


# 2. Conv1D 모델 (Baseline)
class Conv1D(tf.keras.Model):
    def __init__(self, vocab_size, embedd_size, num_classes):
        super(Conv1D, self).__init__()
        self.embedd = tf.keras.layers.Embedding(input_dim=vocab_size,
                                                output_dim=embedd_size)
        self.cnn = tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(64, activation="relu")
        self.outputs = tf.keras.layers.Dense(num_classes, activation="softmax")
        
    def call(self, x):
        x = self.embedd(x)
        x = self.cnn(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.outputs(x)
        return x

# 2-2. RNN 모델 (LSTM 기반)
class RNNModel(tf.keras.Model):
    def __init__(self, vocab_size, embedd_size, num_classes):
        super(RNNModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(input_dim=vocab_size,
                                                   output_dim=embedd_size)
        self.lstm = tf.keras.layers.LSTM(64)
        self.dense = tf.keras.layers.Dense(64, activation='relu')
        self.out = tf.keras.layers.Dense(num_classes, activation='softmax')
    
    def call(self, x):
        x = self.embedding(x)
        x = self.lstm(x)
        x = self.dense(x)
        return self.out(x)

# ---------------------------------------------------------------------------
# 3. 실험 설정: num_words와 모델 종류에 따른 4가지 경우
# ---------------------------------------------------------------------------
experiments = [
    {"num_words": 10000, "model_type": "Conv1D"},
    {"num_words": 10000, "model_type": "RNN"},
    {"num_words": 15000, "model_type": "Conv1D"},
    {"num_words": 15000, "model_type": "RNN"}
]

batch_size = 32
embedd_size = 100
epochs = 5

for exp in experiments:
    num_words_param = exp["num_words"]
    model_type = exp["model_type"]
    
    print("==============================================")
    print(f"실험: num_words = {num_words_param}, 모델 = {model_type}")
    
    # 데이터 로드 (Reuters 데이터는 num_words 인자에 따라 상위 단어만 사용)
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words_param, test_split=0.2)
    
    # 훈련 데이터의 최대 길이 계산
    max_len = np.max(list(map(lambda x: len(x), x_train)))
    print(f"문장의 최대 길이: {max_len}")
    
    # 훈련셋, 테스트셋 패딩
    pad_x_train = pad_sequences(x_train, maxlen=max_len)
    pad_x_test = pad_sequences(x_test, maxlen=max_len)
    
    # y 데이터 reshape (텐서플로우 데이터셋 생성을 위해)
    y_train_reshaped = np.array(y_train).reshape(-1, 1)
    y_test_reshaped = np.array(y_test).reshape(-1, 1)
    
    buffer_size = len(y_train_reshaped)
    
    tf_train = tensorflow_dataset(pad_x_train, y_train_reshaped, buffer_size, batch_size)
    total_num_test = len(pad_x_test)
    tf_test = tensorflow_dataset(pad_x_test, y_test_reshaped, buffer_size, batch_size)
    
    vocab_size = num_words_param  
    num_classes = np.max(y_train) + 1
    
    # 모델 선택: Conv1D 또는 RNN
    if model_type == "Conv1D":
        model = Conv1D(vocab_size, embedd_size, num_classes)
    elif model_type == "RNN":
        model = RNNModel(vocab_size, embedd_size, num_classes)
    else:
        print("모델 타입 오류")
        continue
    
    # 모델 컴파일 및 학습
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    
    model.fit(tf_train,
              steps_per_epoch=buffer_size // batch_size,
              epochs=epochs)
    
    # 모델 평가
    test_loss, test_acc = model.evaluate(tf_test, steps=total_num_test // batch_size)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    
    # 예측 및 F1-score 계산
    y_pred = model.predict(pad_x_test)
    y_pred = np.argmax(y_pred, axis=1)
    
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"정확도: {acc:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(classification_report(y_test, y_pred))

    


실험: num_words = 10000, 모델 = Conv1D
문장의 최대 길이: 2376
Epoch 1/5
280/280 [==============================] - 3s 11ms/step - loss: 1.8117 - accuracy: 0.5545
Epoch 2/5
280/280 [==============================] - 3s 10ms/step - loss: 1.2071 - accuracy: 0.7022
Epoch 3/5
280/280 [==============================] - 3s 11ms/step - loss: 0.8417 - accuracy: 0.7849
Epoch 4/5
280/280 [==============================] - 3s 11ms/step - loss: 0.5669 - accuracy: 0.8554
Epoch 5/5
70/70 [==============================] - 0s 3ms/step - loss: 1.6365 - accuracy: 0.6960
Test Loss: 1.6365
Test Accuracy: 0.6960
정확도: 0.6955
F1-Score: 0.6760
              precision    recall  f1-score   support

           0       0.80      0.33      0.47        12
           1       0.59      0.71      0.65       105
           2       0.18      0.10      0.13        20
           3       0.90      0.90      0.90       813
           4       0.73      0.88      0.80       474
           5       0.00      0.00      0.00         5
    

/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


문장의 최대 길이: 2376
Epoch 1/5
280/280 [==============================] - 20s 63ms/step - loss: 2.2942 - accuracy: 0.4184
Epoch 2/5
280/280 [==============================] - 18s 63ms/step - loss: 1.9368 - accuracy: 0.5015
Epoch 3/5
280/280 [==============================] - 18s 63ms/step - loss: 1.6580 - accuracy: 0.5731
Epoch 4/5
280/280 [==============================] - 18s 63ms/step - loss: 1.5003 - accuracy: 0.6166
Epoch 5/5
70/70 [==============================] - 3s 29ms/step - loss: 1.5689 - accuracy: 0.6036
Test Loss: 1.5689
Test Accuracy: 0.6036
정확도: 0.6046
F1-Score: 0.5664
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.34      0.58      0.43       105
           2       0.00      0.00      0.00        20
           3       0.90      0.95      0.92       813
           4       0.89      0.83      0.86       474
           5       0.00      0.00      0.00         5
           6       0.00      0.00 

/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


문장의 최대 길이: 2376
Epoch 1/5
280/280 [==============================] - 3s 11ms/step - loss: 1.8328 - accuracy: 0.5385
Epoch 2/5
280/280 [==============================] - 3s 11ms/step - loss: 1.1723 - accuracy: 0.7119
Epoch 3/5
280/280 [==============================] - 3s 11ms/step - loss: 0.7848 - accuracy: 0.7954
Epoch 4/5
280/280 [==============================] - 3s 11ms/step - loss: 0.5245 - accuracy: 0.8734
Epoch 5/5
70/70 [==============================] - 0s 3ms/step - loss: 1.5788 - accuracy: 0.7103
Test Loss: 1.5788
Test Accuracy: 0.7103
정확도: 0.7097
F1-Score: 0.6939
              precision    recall  f1-score   support

           0       0.67      0.33      0.44        12
           1       0.60      0.72      0.66       105
           2       0.26      0.35      0.30        20
           3       0.90      0.92      0.91       813
           4       0.78      0.88      0.83       474
           5       0.00      0.00      0.00         5
           6       0.79      0.79      

/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


문장의 최대 길이: 2376
Epoch 1/5
280/280 [==============================] - 20s 64ms/step - loss: 2.3422 - accuracy: 0.4140
Epoch 2/5
280/280 [==============================] - 18s 63ms/step - loss: 1.7970 - accuracy: 0.5408
Epoch 3/5
280/280 [==============================] - 18s 63ms/step - loss: 1.6032 - accuracy: 0.5766
Epoch 4/5
280/280 [==============================] - 18s 63ms/step - loss: 1.4687 - accuracy: 0.6015
Epoch 5/5
70/70 [==============================] - 2s 27ms/step - loss: 1.6040 - accuracy: 0.6125
Test Loss: 1.6040
Test Accuracy: 0.6125
정확도: 0.6126
F1-Score: 0.5860
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.20      0.56      0.30       105
           2       0.00      0.00      0.00        20
           3       0.91      0.93      0.92       813
           4       0.77      0.87      0.82       474
           5       0.00      0.00      0.00         5
           6       0.00      0.00 

/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 결과 분석 

![Sample Image](https://i.imgur.com/ZFEB85I.jpeg)



전반적으로 vocab_size가 증가할수록 대부분의 머신러닝 모델과 딥러닝의 성능이 향상된다. 

최적의 vocab_size는 10,000~15,000으로 보이며, 그 이상에서는 성능 향상이 거의 없는 것으로 보인다.


(1) Naïve Bayes

적은 vocab_size(1,000~3,000)에서도 안정적이지만 성능이 낮음. vocab_size 증가에 따른 성능 향상이 크지 않음
각 단어가 개별적으로 작용한다고 가정하는 단순한 모델

(2) Complement Naïve Bayes

일반 NB보다 성능이 뛰어나며, vocab_size 증가에 따라 성능이 꾸준히 증가됨
각 클래스에서 단어 출현 빈도를 보정했기때문에 성능이 더 뛰어남

(3) Logistic Regression

vocab_size 증가에 따라 큰 성능 향상이 보임. 
더 많은 단어(feature)를 사용할 수 있어 학습이 더 정밀해진듯

(4) Support Vector Classifier 

vocab_size 증가에 따라 큰 성능 향상이 보임.
역시다 더 많은 단어를 사용할 수 있어 학습이 정밀해짐

(5) Decision Tree 

vocab_size 증가에 따라 성능이 약간 감소. 
단어(feature)가 많아지면서 트리가 너무 복잡해지고, 불필요한 분할이 많아짐 
차원의 저주때문 

(6) Random Forest 

성능이 일정 수준 이상으로 유지되며, 10,000 vocab에서 최고 성능을 보임
일정 이상(15,000~20,000)에서는 더 이상 정보 이득이 크지 않아서 성능이 정체되는듯

(7) Gradient Boosting Tree

트리 기반 모델 중 가장 좋은 성능을 보이며, vocab_size 증가에 따라 꾸준한 향상
부스팅 기법임


(8) Voting

가장 높은 성능을 기록하며, vocab_size 증가에 따라 꾸준히 향상됨. 앙상블 학습.


(9) Conv1D

 vocab_size 증가에 따라서 꾸준한 향상, 
딥러닝 모델은 많은 데이터를 필요로 하며, vocab_size가 증가할수록 성능이 크게 향상됨.



## 회고

코드들이 너무 돌아가는데 엄청 오래걸려서.. 

팀원들과 같이 vocab 사이즈를 정하고 해서 모델, vocab 사이즈간 성능비교가 쉬웠음
